In [ ]:
#default_exp pleiades

# Pleiades

> Working through conversion to cogs, upload to object storage, stac item metadata parsing, etc. for pleiades datasets

In [1]:
#hide
%load_ext autoreload
%autoreload 2

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import os
from glob import glob
import time
import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import STAC_IO
from pystac.extensions.eo import Band
import geopandas as gpd

from sac_stac.utils import sedas_client, sedas_find_datasets, sedas_download, sedas_extract
from sac_stac.utils import cogmosaicbands
from sac_stac.utils import s3_upload_dir, s3_list_objects_paths, clean_up
from sac_stac.utils import pystac_setIO, create_uri

In [4]:
import pandas as pd

## **Preparation**: ***Download, cloud-optimise, upload***

### **Prep Function**

Download and use basic gdal tools to mosaic any tiles into single images, convert to cog and upload to our object storage.

In [ ]:
#export
def prep_pleiades(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="uksa-ssgp/pleiades/"):
    try:
        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{scene_name}.zip"
        scene_dir = f"{down_zip[:-4]}/"
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, scene_dir)
        # sensor-specific band mosaicing and cogifying
        imgs_ms = glob(f"{scene_dir}*/*MS_002*/*.TIF")
        imgs_pan = glob(f"{scene_dir}*/*P_001*/*.TIF")
        cogmosaicbands(imgs_pan, 1, imgs_pan[0][:-13])
        cogmosaicbands(imgs_ms, 4, imgs_ms[0][:-13])
        # upload
        s3_upload_dir(scene_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {scene_name} at {s3_dir}{scene_name}/")
        clean_up(inter_dir)
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
        clean_up(inter_dir)

### **Iteration with examples**

Some samples used to iterate creation of the prep function.

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products']).head(2)

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,c0ec9a5f12356e87bc910ddbc49dbb76,Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.654728 51.309517,-1.345414 51.3081...",2018-10-24T11:17:22Z,...,4.825125e+08,1.0,5.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'cf...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,94cc7887414be0912de7ca44288f79da,Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-2.153852 51.603313,-1.841533 51.6033...",2018-09-29T11:10:08Z,...,1.565260e+08,2.0,22.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


Example of testing the first one.

In [ ]:
done = ["Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802241127550_FR1_PX_W002N51_0711_024843613631101",
        "Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101",
        "Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_201807241124303_FR1_PX_W002N51_0720_01561TPP1601069279",
        "Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201809021117346_FR1_PX_W002N51_1008_03279TPP1601069492",
        "Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318",
        "Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712"]

In [ ]:
#hide
# for p in pd.DataFrame(result['products']).supplierId.values:
    
#     if not True in [i.split('_')[1] in p for i in done]:
#         prep_pleiades(p)

In [ ]:
# prep_pleiades("Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712")

### **Full Job Lists**

Once happy with `prep_pleiades` we create a list of jobs from search results of `sedas_find_datasets` to submit to a redis queue as per `rediswq`.

We can test these via redis within `pleiades_prep_worker`.

Sometimes we run as campaigns via K8s.

In [ ]:
jobs_dir = "/tmp/data/"
td = datetime.today()
td = td.strftime('%Y')+td.strftime('%m')+td.strftime('%d')
td

'20201111'

In [ ]:
with open(f"{jobs_dir}/JOBLIST_{td}_pleiades.txt", 'w') as t:
    for v in pd.DataFrame(result['products']).supplierId.values:
        t.write("rpush jobPL"+" '{"+'"sedas_supplierId": "'+ v +'"'"}'" + '\n')

## **STAC metadata**: ***core & extensions***

With the cogs above hosted object storage we can go create some tools for building a STAC *Collection* from them, to be used within in **insert nb** to build a *Catalog* of different Catapult *Collections*. As per the nb these initially sit within a *static* STAC alongside the actual datasets on the object storage. However we plan on hosting via a STAC compliant API - probs [pygeoapi](https://pygeoapi.io/).

In [6]:
obj_paths_list = s3_list_objects_paths('public-eo-data', 'uksa-ssgp/pleiades/')

In [7]:
# get unique Item / scene names (third dir from path)
scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
scene_names[:5]

['Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101',
 'Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201809291110083_FR1_PX_W002N51_0116_00596TPP1601069431',
 'Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_201802251121249_FR1_PX_W002N51_0314_01048',
 'Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_201802251121249_FR1_PX_W002N51_0314_010483613775101',
 'Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_201807241124303_FR1_PX_W002N51_0720_01561TPP1601069279']

### **Individual example for iteration**

can just work with one

In [12]:
scene_name = scene_names[0]
scene_name

'Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101'

and objects associated with that scene

In [13]:
scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
scene_obj_paths[:2]

['uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/DELIVERY.PDF',
 'uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/DIM_PHR1A_MS_201809291110008_ORT_3614104101-2.XML']

set our own I/O for pystac

In [14]:
#export
pystac_setIO()

### **Functions for** ***Item*** **metadata**

In [15]:
#export
def pleiades_get_dt(scene_name):
    return datetime.strptime(scene_name.split('_')[5][:14], '%Y%m%d%H%M%S')

In [16]:
pleiades_get_dt(scene_name)

datetime.datetime(2018, 9, 29, 11, 10)

In [22]:
#export
def pleiades_parsemeta(scene_name, scene_obj_paths):
    meta_path = [i for i in scene_obj_paths if (i.endswith('.XML')) & (os.path.basename(i).startswith('DIM'))][0]
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

In [23]:
meta = pleiades_parsemeta(scene_name, scene_obj_paths)
meta['Dimap_Document'].keys()

odict_keys(['Metadata_Identification', 'Dataset_Identification', 'Dataset_Content', 'Product_Information', 'Coordinate_Reference_System', 'Geoposition', 'Processing_Information', 'Raster_Data', 'Radiometric_Data', 'Geometric_Data', 'Quality_Assessment', 'Dataset_Sources'])

In [67]:
#export
def pleiades_get_crs(metadata):
    return int(metadata['Dimap_Document']['Coordinate_Reference_System']['Projected_CRS']['PROJECTED_CRS_NAME'])

In [37]:
crs = pleiades_get_crs(meta)
crs

27700

In [17]:
#export
def pleiades_get_geom(scene_paths):
    roi_path = [i for i in scene_paths if (i.endswith('1_MSK.GML') * os.path.basename(i).startswith('ROI'))][0]
    roi_uri = create_uri(roi_path)
    return json.loads(gpd.read_file(roi_uri).to_crs('EPSG:4326').to_json(show_bbox=True))['features'][0]['geometry']

In [18]:
pleiades_get_geom(scene_obj_paths)

{'type': 'Polygon',
 'coordinates': [[[-1.7162111146081491, 51.703447752207374],
   [-1.6539393674716631, 51.70305171212088],
   [-1.5922000556276548, 51.7022918404328],
   [-1.592478005824943, 51.66971308516982],
   [-1.5923937686249074, 51.660564184797174],
   [-1.5924483796708082, 51.65689143996495],
   [-1.5923435803877832, 51.6484257860413],
   [-1.5925225669770522, 51.629252456312805],
   [-1.6313240498998447, 51.629686309100045],
   [-1.6617769179346535, 51.62994873863878],
   [-1.715447257136181, 51.63018534903567],
   [-1.7483946591920845, 51.630161588463274],
   [-1.7789150489058498, 51.63009707066823],
   [-1.8369443638514822, 51.62966137158252],
   [-1.8706869752457782, 51.629244747190995],
   [-1.902984365251201, 51.62878127998801],
   [-1.9031756192477154, 51.64416997236793],
   [-1.9032281966762568, 51.65003679955556],
   [-1.9031830688830722, 51.650868451669226],
   [-1.9032398631646559, 51.65134952934474],
   [-1.9032635080951406, 51.6606374853309],
   [-1.903368845801

In [26]:
#export
def pleiades_get_bbox(metadata):
    lons = [float(i['LON']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    lats = [float(i['LAT']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    return [min(lons), min(lats), max(lons), max(lats)]

In [27]:
pleiades_get_bbox(meta)

[-1.903199885312827, 51.62869184186537, -1.592242985175717, 51.70382196570706]

In [66]:
#export
def pleiades_get_gsd(metadata):
    across = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ACROSS_TRACK']['#text'])
    along = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ALONG_TRACK']['#text'])
    return round(( across + along ) / 2, 2)

In [30]:
pleiades_get_gsd(meta)

0.74

In [65]:
#export
def pleiades_get_cloudcover(metadata):
    return round(float(metadata['Dimap_Document']['Dataset_Content']['CLOUD_COVERAGE']['#text']),2)

In [32]:
pleiades_get_cloudcover(meta)

0.0

**Can now create** ***Item*** **containing core STAC metadata.**

In [40]:
def pleiades_create_item(scene_name, scene_obj_paths):

    meta = pleiades_parsemeta(scene_name, scene_obj_paths)

    crs = pleiades_get_crs(meta)

    item = pystac.Item(id=scene_name,
                      datetime=pleiades_get_dt(scene_name),
                      geometry=pleiades_get_geom(scene_obj_paths),
                      bbox=pleiades_get_bbox(meta),
                      properties={})

    item.common_metadata.gsd = pleiades_get_gsd(meta)

    item.ext.enable('eo')
    item.ext.eo.cloud_cover = pleiades_get_cloudcover(meta)

    item.ext.enable('projection')
    item.ext.projection.epsg = pleiades_get_crs(meta)

    return item

In [41]:
example_item = pleiades_create_item(scene_name, scene_obj_paths)
example_item.validate()

In [42]:
example_item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101',
 'properties': {'gsd': 0.74,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700,
  'datetime': '2018-09-29T11:10:00Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.7162111146081491, 51.703447752207374],
    [-1.6539393674716631, 51.70305171212088],
    [-1.5922000556276548, 51.7022918404328],
    [-1.592478005824943, 51.66971308516982],
    [-1.5923937686249074, 51.660564184797174],
    [-1.5924483796708082, 51.65689143996495],
    [-1.5923435803877832, 51.6484257860413],
    [-1.5925225669770522, 51.629252456312805],
    [-1.6313240498998447, 51.629686309100045],
    [-1.6617769179346535, 51.62994873863878],
    [-1.715447257136181, 51.63018534903567],
    [-1.7483946591920845, 51.630161588463274],
    [-1.7789150489058498, 51.63009707066823],
    [-1.8369443638514822, 51.62966137158252],
    [-1.8706869752457782, 51.62924

### **Functions for** ***Asset*** **metadata**

Once we have *Item* level metadata we can add the actual *Assets*. There are a few constants used within these functions at the *Item* level (i.e. band info) and criteria for finding within the object paths.

In [44]:
#export
pleiades_bands = [Band.create(name='Panchromatic', description='Panchromatic: 480 - 830 nm', common_name='pan'),
                  Band.create(name='Blue', description='Blue: 430 - 550 nm', common_name='blue'),
                  Band.create(name='Green', description='Green: 490 - 610 nm', common_name='green'),
                  Band.create(name='Red', description='Red: 600 - 720 nm', common_name='red'),
                  Band.create(name='Near-Infrared', description='Near-Infrared: 750 - 950 nm', common_name='nir')]

In [68]:
#export
pleiades_band_refs = {
    'Panchromatic':{'ends':'_band1', 'dif':'_P_', 'id':'B0'},
    'Blue':{'ends':'_band1', 'dif':'_MS_', 'id':'B1'},
    'Green':{'ends':'_band2', 'dif':'_MS_', 'id':'B2'},
    'Red':{'ends':'_band3', 'dif':'_MS_', 'id':'B3'},
    'Near-Infrared':{'ends':'_band4', 'dif':'_MS_', 'id':'B4'}
}

In [69]:
pleiades_band_refs

{'Panchromatic': {'ends': '_band1', 'dif': '_P_', 'id': 'B0'},
 'Blue': {'ends': '_band1', 'dif': '_MS_', 'id': 'B1'},
 'Green': {'ends': '_band2', 'dif': '_MS_', 'id': 'B2'},
 'Red': {'ends': '_band3', 'dif': '_MS_', 'id': 'B3'},
 'Near-Infrared': {'ends': '_band4', 'dif': '_MS_', 'id': 'B4'}}

Need to find the actual asset path for a given band. (Note that we typically store COGs per-band.)

In [47]:
def pleiades_find_band_path(band_name, scene_obj_paths):
    matched_paths = [ o for o in scene_obj_paths if (o.endswith(f"{pleiades_band_refs[band_name]['ends']}.tif")) & (f"{pleiades_band_refs[band_name]['dif']}" in o) ]
    if len(matched_paths) > 1:
        raise Exception(f"Found too many matches: {matched_paths}")
#     elif len(matched_paths) == 0: # should probably add something for when no asset is found...
#         raise Warning(f"")
    return matched_paths[0]

In [48]:
pleiades_find_band_path('Blue', scene_obj_paths)

'uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/IMG_PHR1A_MS_201809291110008_ORT_36141041_band1.tif'

Then we can add each asset expected to be found and test with the example item taken from above.

In [49]:
def pleiades_add_assets2item(item, scene_obj_paths):
    for band in pleiades_bands:
#         print(band.name)

        band_path = pleiades_find_band_path(band.name, scene_obj_paths)
        band_url = create_uri(band_path)
#         print(band_url)

        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        item.ext.eo.set_bands([band], asset)
        item.add_asset(pleiades_band_refs[band.name]['id'], asset)    
        
    return item

In [51]:
example_item_with_assets = pleiades_add_assets2item(example_item, scene_obj_paths)

Now we can see the completed *Item* stac record.

In [52]:
example_item_with_assets.validate()

In [53]:
example_item_with_assets.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101',
 'properties': {'gsd': 0.74,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700,
  'datetime': '2018-09-29T11:10:00Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.7162111146081491, 51.703447752207374],
    [-1.6539393674716631, 51.70305171212088],
    [-1.5922000556276548, 51.7022918404328],
    [-1.592478005824943, 51.66971308516982],
    [-1.5923937686249074, 51.660564184797174],
    [-1.5924483796708082, 51.65689143996495],
    [-1.5923435803877832, 51.6484257860413],
    [-1.5925225669770522, 51.629252456312805],
    [-1.6313240498998447, 51.629686309100045],
    [-1.6617769179346535, 51.62994873863878],
    [-1.715447257136181, 51.63018534903567],
    [-1.7483946591920845, 51.630161588463274],
    [-1.7789150489058498, 51.63009707066823],
    [-1.8369443638514822, 51.62966137158252],
    [-1.8706869752457782, 51.62924

### **Compiling** ***Items*** **into a** ***Collection*** 

We naurally want to apply the above tools to all Pleiades-related *Items* and their *Assets* in order to build a *Collection* that can sit within another *Collection* i.e. comprised of other UKSA/SSGP-procured datasets and maybe an overall Satellite Applications Catapult *Catalog* covering all of our internal and external-facing geospatial datasets.

In [60]:
def pleiades_create_collection(pleiades_dir, bucket='public-eo-data'):
    
    collection_id = 'uksa-ssgp-pleiades'
    collection_title = 'SSGP-procured Pleiades images over the UK'
    collection_description = '''### UKSA / SSGP Pleiades

    A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).
    '''
    
    # initially arbitrary as updated later
    spatial_extent = pystac.SpatialExtent([[-7.57216793459, 49.959999905, 1.68153079591, 58.6350001085]])
    temporal_extent = pystac.TemporalExtent([[datetime(2011, 12, 16), None]])
    collection_extent = pystac.Extent(spatial_extent, temporal_extent)
    
    collection = pystac.Collection(id=collection_id,
                                   title=collection_title,
                                   description=collection_description,
                                   extent=collection_extent)
    
    collection.providers = [
        pystac.Provider(name='Airbus Defence & Space', roles=['producer'], url='https://www.airbus.com/space.html'),
        pystac.Provider(name='UK Space Agency', roles=['licensor'], url='https://www.gov.uk/government/organisations/uk-space-agency'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['processor'], url='https://sa.catapult.org.uk/'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['host'], url='https://sa.catapult.org.uk/')
    ]
    
    obj_paths_list = s3_list_objects_paths(bucket, pleiades_dir)
    scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
    
    for scene_name in scene_names:
        
        scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
        
        item = pleiades_create_item(scene_name, scene_obj_paths)
        item = pleiades_add_assets2item(item, scene_obj_paths)
        
        collection.add_item(item)
    
    collection.update_extent_from_items()
        
    return collection
    

In [61]:
example_collection = pleiades_create_collection('uksa-ssgp/pleiades/')

In [64]:
example_collection.describe()

* <Collection id=uksa-ssgp-pleiades>
  * <Item id=Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101>
  * <Item id=Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201809291110083_FR1_PX_W002N51_0116_00596TPP1601069431>
  * <Item id=Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_201802251121249_FR1_PX_W002N51_0314_010483613775101>
  * <Item id=Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_201807241124303_FR1_PX_W002N51_0720_01561TPP1601069279>
  * <Item id=Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201809021117346_FR1_PX_W002N51_1008_03279TPP1601069492>
  * <Item id=Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318>
  * <Item id=Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802241127550_FR1_PX_W002N51_0711_024843613631101>
  * <Item id=Pleiades_UKSA87_SO18034613-87-01_DS_PHR1A_201806291118170_FR1_PX_W002N51_1016_029773614015101>


In [63]:
example_collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\n    A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n    ',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-2.153701453474465,
     51.15439056920768,
     -1.082452236576131,
     51.90716967216828]]},
  'temporal': {'in

Note: We normalised hrefs and save in a different notebook.

## Export

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_rediswq.ipynb.
Converted 00_utils.ipynb.
Converted 01A_pleiades.ipynb.
Converted 01B_pleiades_prep_worker.ipynb.
Converted 02A_spot.ipynb.
Converted index.ipynb.
